In [1]:
out_mix_path = 'demo/mix.mp4'
crf = 24

label_path_list = ['real-time-raw-label.npz']
ckpt_path = 'ckpt/R50_DeAOTL_PRE_YTB_DAV.pth'
model = 'r50_deaotl'


DEVICE = 'mps' # cuda

In [2]:

cfg = {
    'phase': 'PRE_YTB_DAV',
    'model': model,
    'model_path': ckpt_path,
    'long_term_mem_gap': 99999,
    'max_len_long_term': 30,
    'device': DEVICE,
}

In [3]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image

import numpy as np
from tqdm import tqdm
from vos.utils.plot import gen_mix
from vos.utils.avio import ReadArray, WriteArray
from vos.tracker import get_aot

In [4]:
# # record 100 frame for build ROI mask
# target_n = 100
# export = WriteArray(out_path, 30., crf)
# cap = cv2.VideoCapture(0)
# cap.set(3,640) # adjust width
# cap.set(4,480) # adjust height

# index = 0
# while True:
#     success, img = cap.read()
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     export.append(img_rgb)
#     cv2.imshow("Webcam", img) # This will open an independent window
#     index += 1


#     if index == target_n:
#         break
#     if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
#         cap.release()
#         break
        
# cv2.destroyAllWindows() 
# cv2.waitKey(1) #
# export.close()


In [5]:
tracker = get_aot(cfg)
frames = [np.load(it)['frame'] for it in label_path_list]
masks = [np.load(it)['mask'] for it in label_path_list]
tracker.add_reference_frame(frames[0], masks[0], 1, -1)


cap = cv2.VideoCapture(0)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = tracker.track(img)
    mask = mask.squeeze().detach().cpu().numpy().astype(np.uint8)
    mix = gen_mix(img, mask)
    mix_bgr = cv2.cvtColor(mix, cv2.COLOR_BGR2RGB)
    cv2.imshow("Webcam", mix_bgr) # This will open an independent window
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
        
cv2.destroyAllWindows() 
cv2.waitKey(1) #

AOTTracker Device: mps


/Users/raiso/VOS_DeAOT/.venv/lib/python3.11/site-packages/torch/nn/functional.py:4817: UserWarning: The operator 'aten::im2col' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._C._nn.im2col(input, _pair(kernel_size), _pair(dilation), _pair(padding), _pair(stride))
2024-05-20 23:14:10.974 Python[29070:565024] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


-1